<a href="https://colab.research.google.com/github/Skynetplus0/HAWKEYE_Yolo-v11_Model_for_Object_Detection_and_Tracking_on_UAV/blob/main/HAWKEYE_Yolov11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install ultralytics==8.*
!yolo checks






   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/112.6 GB disk)

OS                     Linux-6.6.105+-x86_64-with-glibc2.35
Environment            Colab
Python                 3.12.12
Install                pip
Path                   /usr/local/lib/python3.12/dist-packages/ultralytics
RAM                    12.67 GB
Disk                   38.7/112.6 GB
CPU                    Intel Xeon CPU @ 2.00GHz
CPU count              2
GPU                    Tesla T4, 15095MiB
GPU count              1
CUDA                   12.6

numpy      

In [9]:
from google.colab import drive
drive.mount('/content/drive')

# ---- DRIVE'DAKİ KLASÖR YAPIN ----
DATASET_DIR = "/content/drive/MyDrive/dataset"   # dataset klasörü
CFG_DIR     = "/content/drive/MyDrive/cfg"       # yaml'ler
RUNS_DIR    = "/content/drive/MyDrive/runs"      # eğitim çıktıları

import os
os.makedirs(CFG_DIR, exist_ok=True)
os.makedirs(RUNS_DIR, exist_ok=True)

print("DATASET_DIR:", DATASET_DIR)
print("CFG_DIR:", CFG_DIR)
print("RUNS_DIR:", RUNS_DIR)

!ls -la "{DATASET_DIR}"
!ls -la "{DATASET_DIR}/images"
!ls -la "{DATASET_DIR}/labels"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DATASET_DIR: /content/drive/MyDrive/dataset
CFG_DIR: /content/drive/MyDrive/cfg
RUNS_DIR: /content/drive/MyDrive/runs
total 9
-rw------- 1 root root  254 Jan 26 18:08 data.yaml
drwx------ 5 root root 4096 Jan 26 17:58 images
drwx------ 5 root root 4096 Jan 26 17:58 labels
total 12
drwx------ 2 root root 4096 Jan 26 17:58 test
drwx------ 2 root root 4096 Jan 26 17:58 train
drwx------ 2 root root 4096 Jan 26 17:58 valid
total 12
drwx------ 2 root root 4096 Jan 26 17:58 test
drwx------ 2 root root 4096 Jan 26 17:58 train
drwx------ 2 root root 4096 Jan 26 17:58 valid


In [10]:
import os

required_dirs = [
    f"{DATASET_DIR}/images/train",
    f"{DATASET_DIR}/images/valid",
    f"{DATASET_DIR}/images/test",
    f"{DATASET_DIR}/labels/train",
    f"{DATASET_DIR}/labels/valid",
    f"{DATASET_DIR}/labels/test",
]

missing = [d for d in required_dirs if not os.path.isdir(d)]
if missing:
    print("❌ Eksik klasörler:")
    for d in missing:
        print(" -", d)
    raise SystemExit("Dataset klasör yapısı eksik. Klasör adlarını kontrol et.")
else:
    print("✅ Dataset klasör yapısı OK")

def count_files(path):
    return len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])

print("images/train:", count_files(f"{DATASET_DIR}/images/train"))
print("labels/train:", count_files(f"{DATASET_DIR}/labels/train"))
print("images/valid:", count_files(f"{DATASET_DIR}/images/valid"))
print("labels/valid:", count_files(f"{DATASET_DIR}/labels/valid"))
print("images/test :", count_files(f"{DATASET_DIR}/images/test"))
print("labels/test :", count_files(f"{DATASET_DIR}/labels/test"))

✅ Dataset klasör yapısı OK
images/train: 558
labels/train: 558
images/valid: 84
labels/valid: 84
images/test : 46
labels/test : 46


In [11]:
DATA_YAML = f"{CFG_DIR}/data.yaml"

data_yaml_text = f"""path: {DATASET_DIR}
train: images/train
val: images/valid
test: images/test

names:
  0: drone
"""

with open(DATA_YAML, "w") as f:
    f.write(data_yaml_text)

print("✅ Saved:", DATA_YAML)
!cat "{DATA_YAML}"

✅ Saved: /content/drive/MyDrive/cfg/data.yaml
path: /content/drive/MyDrive/dataset
train: images/train
val: images/valid
test: images/test

names:
  0: drone


In [16]:
MODEL_BASE = f"{CFG_DIR}/yolo_model.yaml"
MODEL_S    = f"{CFG_DIR}/yolo_model_s.yaml"

yolo_model_yaml = r"""
# yolo_model.yaml
# YOLO11 UAV/Drone detection modeli (P2-P5) ölçekli
# =================================================

nc: 1

#scales:
 # n: [0.33, 0.25, 1024]
 # s: [0.33, 0.50, 1024]
 # m: [0.67, 0.75, 768]
 # l: [1.00, 1.00, 512]
 # x: [1.00, 1.25, 512]

depth_multiple: 0.33
width_multiple: 0.50
max_channels: 1024

backbone:
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [-1, 3, C2f, [128, True]]

  - [-1, 1, Conv, [256, 3, 2]]
  - [-1, 6, C2f, [256, True]]

  - [-1, 1, Conv, [512, 3, 2]]
  - [-1, 6, C2f, [512, True]]

  - [-1, 1, Conv, [1024, 3, 2]]
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]

head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]]
  - [-1, 3, C2f, [256]]

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 2], 1, Concat, [1]]
  - [-1, 3, C2f, [128]]

  - [-1, 1, Conv, [128, 3, 2]]
  - [[-1, 15], 1, Concat, [1]]
  - [-1, 3, C2f, [256]]

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]
  - [-1, 3, C2f, [512]]

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]
  - [-1, 3, C2f, [1024]]

  - [[18, 21, 24, 27], 1, Detect, [nc]]
""".strip()

with open(MODEL_BASE, "w") as f:
    f.write(yolo_model_yaml)

# Small ölçeği daha sağlam seçebilmek için kopya
import shutil
shutil.copy(MODEL_BASE, MODEL_S)

print("✅ Saved:", MODEL_BASE)
print("✅ Saved:", MODEL_S)
!ls -la "{CFG_DIR}"

✅ Saved: /content/drive/MyDrive/cfg/yolo_model.yaml
✅ Saved: /content/drive/MyDrive/cfg/yolo_model_s.yaml
total 4
-rw------- 1 root root  112 Jan 26 18:39 data.yaml
-rw------- 1 root root 1257 Jan 26 18:46 yolo_model_s.yaml
-rw------- 1 root root 1257 Jan 26 18:46 yolo_model.yaml


In [17]:
from ultralytics import YOLO

model = YOLO(MODEL_S)

train_name = "hawkeye_yolo11_p2p5_small"

results = model.train(
    data=DATA_YAML,     # ✅ MUTLAKA YAML DOSYASI
   # scale="s",          # ✅ SMALL ZORLA (no scale warning bitmeli)
    epochs=150,
    imgsz=640,          # küçük objeler için iyi; OOM olursa 768/640 yap
    batch=-1,           # auto batch
    device=0,
    workers=2,
    project=RUNS_DIR,
    name=train_name,

    optimizer="SGD",
    lr0=0.01,
    cos_lr=True,
    warmup_epochs=3.0,
    patience=50,
    close_mosaic=10,

    # augment
    degrees=5.0,
    translate=0.10,
    scale=0.50,
    fliplr=0.5,

    amp=True
)

print("✅ Train finished:", f"{RUNS_DIR}/{train_name}")

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/cfg/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/cfg/yolo_model_s.yaml, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=hawkeye_yolo11_p2p5_small5, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, ov

In [20]:
from ultralytics import YOLO

#best_path = f"{RUNS_DIR}/{train_name}/weights/best.pt"
#last_path = f"{RUNS_DIR}/{train_name}/weights/last.pt"
best_path = f"{RUNS_DIR}/hawkeye_yolo11_p2p5_small5/weights/best.pt"
last_path = f"{RUNS_DIR}/hawkeye_yolo11_p2p5_small5/weights/last.pt"



print("best:", best_path)
print("last:", last_path)

model = YOLO(best_path)
metrics_val = model.val(data=DATA_YAML, imgsz=640, device=0)
print(metrics_val)

best: /content/drive/MyDrive/runs/hawkeye_yolo11_p2p5_small5/weights/best.pt
last: /content/drive/MyDrive/runs/hawkeye_yolo11_p2p5_small5/weights/last.pt
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO_model_s summary (fused): 91 layers, 10,626,708 parameters, 0 gradients, 36.6 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.3 ms, read: 49.2±15.9 MB/s, size: 83.2 KB)
val: Scanning /content/drive/MyDrive/dataset/labels/valid.cache... 84 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 84/84 25.2Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 2.1it/s 2.9s
                   all         84         89      0.986      0.966      0.978      0.726
Speed: 8.3ms preprocess, 13.8ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /content/runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.m

In [21]:
from ultralytics import YOLO

#best_path = f"{RUNS_DIR}/{train_name}/weights/best.pt"

best_path = f"{RUNS_DIR}/hawkeye_yolo11_p2p5_small5/weights/best.pt"
model = YOLO(best_path)

metrics_test = model.val(data=DATA_YAML, split="test", imgsz=640, device=0)
print(metrics_test)


Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO_model_s summary (fused): 91 layers, 10,626,708 parameters, 0 gradients, 36.6 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.2±0.0 MB/s, size: 82.7 KB)
val: Scanning /content/drive/MyDrive/dataset/labels/test... 46 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 46/46 1.2it/s 37.2s
val: New cache created: /content/drive/MyDrive/dataset/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.5it/s 2.0s
                   all         46         47      0.971          1      0.994      0.751
Speed: 8.1ms preprocess, 15.8ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to /content/runs/detect/val2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object a

In [22]:
from ultralytics import YOLO

#best_path = f"{RUNS_DIR}/{train_name}/weights/best.pt"

best_path = f"{RUNS_DIR}/hawkeye_yolo11_p2p5_small5/weights/best.pt"
model = YOLO(best_path)

TEST_SOURCE = f"{DATASET_DIR}/images/test"

pred = model.predict(
    source=TEST_SOURCE,
    imgsz=640,
    conf=0.25,
    iou=0.6,
    device=0,
    save=True,
    project=RUNS_DIR,
    name=f"{train_name}_pred_test"
)

print("✅ Predictions saved to:", f"{RUNS_DIR}/{train_name}_pred_test")



image 1/46 /content/drive/MyDrive/dataset/images/test/106_JPEG.rf.9159e45f540749a872c1351706a139ee.jpg: 640x640 2 drones, 20.6ms
image 2/46 /content/drive/MyDrive/dataset/images/test/108_JPEG.rf.fc58f293a1bb191cd1f23e6192357015.jpg: 640x640 1 drone, 20.5ms
image 3/46 /content/drive/MyDrive/dataset/images/test/120_JPEG_jpg.rf.16bfb2070fa674886ecdd2158d004163.jpg: 640x640 1 drone, 20.5ms
image 4/46 /content/drive/MyDrive/dataset/images/test/135_JPEG_jpg.rf.52825a3dfe0207af89aebb25ae7c43a9.jpg: 640x640 1 drone, 20.5ms
image 5/46 /content/drive/MyDrive/dataset/images/test/142_JPEG.rf.c65972dcef30b6246ceff9b1d4c54ed7.jpg: 640x640 1 drone, 20.5ms
image 6/46 /content/drive/MyDrive/dataset/images/test/157_JPEG.rf.07b50342b06598614ef48ab84f817ec9.jpg: 640x640 1 drone, 20.5ms
image 7/46 /content/drive/MyDrive/dataset/images/test/173_JPEG.rf.71e58762811f4bc4e0510db38da46dce.jpg: 640x640 1 drone, 16.5ms
image 8/46 /content/drive/MyDrive/dataset/images/test/18_JPEG.rf.d526ac98f941a30164905b0e6d1a3

In [23]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/runs/hawkeye_yolo11_p2p5_small5/weights/best.pt")

model.export(
    format="onnx",
    imgsz=640,
    opset=12,
    simplify=True
)

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO_model_s summary (fused): 91 layers, 10,626,708 parameters, 0 gradients, 36.6 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/runs/hawkeye_yolo11_p2p5_small5/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 34000) (20.7 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<2.0.0', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 208ms
Prepared 6 packages in 6.96s
Installed 6 packages in 248ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.20.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 8.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updat

'/content/drive/MyDrive/runs/hawkeye_yolo11_p2p5_small5/weights/best.onnx'